In [ ]:
# 11-1

In [ ]:
# 11-2

In [3]:
# 11-3

class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

f = Foo()
print(f[1])
for i in f: print(i)
print(20 in f)
print(15 in f)

10
0
10
20
True
False


In [6]:
# 11-4

import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list("JQKA")
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [8]:
# 11-5

from random import shuffle

deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [9]:
# 11-6

def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]



[Card(rank='2', suit='spades'),
 Card(rank='9', suit='diamonds'),
 Card(rank='4', suit='diamonds'),
 Card(rank='J', suit='diamonds'),
 Card(rank='A', suit='diamonds')]

In [ ]:
# 11-7

try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)

In [15]:
# 11-8

import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # <1>
        self._cards[position] = value

    def __delitem__(self, position):  # <2>
        del self._cards[position]

    def insert(self, position, value):  # <3>
        self._cards.insert(position, value)
        
f = FrenchDeck2()


In [17]:
# 11-9

# BEGIN TOMBOLA_ABC

import abc

class Tombola(abc.ABC):  # <1>

    @abc.abstractmethod
    def load(self, iterable):  # <2>
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # <5>


    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(sorted(items))


# END TOMBOLA_ABC

In [ ]:
# 11-10

# 책 참고

In [19]:
# 11-11

class Fake(Tombola):
    def pick(self):
        return 13
    
print(Fake)
f = Fake()

<class '__main__.Fake'>


TypeError: Can't instantiate abstract class Fake with abstract method load

In [21]:
# 11-12

import random

class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [22]:
# 11-13

class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

In [23]:
# 11-14

from random import randrange

@Tombola.register  # <1>
class TomboList(list):  # <2>

    def pick(self):
        if self:  # <3>
            position = randrange(len(self))
            return self.pop(position)  # <4>
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # <5>

    def loaded(self):
        return bool(self)  # <6>

    def inspect(self):
        return tuple(sorted(self))

# Tombola.register(TomboList)  # <7>


In [25]:
print(issubclass(TomboList, Tombola))

t = TomboList(range(100))

isinstance(t, Tombola)

True


True

In [26]:
TomboList.__mro__

(__main__.TomboList, list, object)

In [32]:
# 11-15

import doctest

TEST_FILE = "/Users/kjs/Desktop/fluent_python/src/11-iface-abc/tombola_tests.rst"
TEST_MSG = '{0:16} {1.attempted:2} tests, {1.failed:2} failed - {2}'

def main(argv):
    verbose = '-v' in argv
    real_subclasses = Tombola.__subclasses__()
    virtual_subclasses = list(Tombola._dump_registry())

    for cls in real_subclasses + virtual_subclasses:
        test(cls, verbose)

def test(cls, verbose=False):
    res = doctest.testfile(
        TEST_FILE,
        globs={'ConcreteTombola': cls},
        verbose=verbose,
        optionflags=doctest.REPORT_ONLY_FIRST_FAILURE)
    tag = 'FAIL' if res.failed else 'OK'
    print(TEST_MSG.format(cls.__name__, res, tag))

import sys
main(sys.argv)

Class: __main__.Tombola
Inv. counter: 49
_abc_registry: {<weakref at 0x1077fd0d0; to 'type' at 0x10b0c2b30 (TomboList)>}
_abc_cache: set()
_abc_negative_cache: set()
_abc_negative_cache_version: 49


AttributeError: 'NoneType' object has no attribute '_abc_register'

In [ ]:
# 11-16, 11-17 생략